In [1]:
import sys
import csv
import numpy as np
import string

from scipy import stats
from matplotlib import pyplot
import time
from IPython import display

In [2]:
# !{sys.executable} -m pip install --user gensim
import gensim

### Train & Test data Loading

In [3]:
train_data  = list(csv.reader(open('./train.txt', encoding="utf8")))
test_data  = list(csv.reader(open('./test.txt', encoding="utf8")))
stop_words  = list(csv.reader(open('./stopwords.txt', encoding="utf8")))
stop_words = [row[0] for row in  stop_words ]

In [4]:
def removePunctuation(text):
    text_to_return = ""
    for char in text:
        if char not in string.punctuation:
            text_to_return = text_to_return + char
    return text_to_return 

In [5]:
def removeStopWords(text):
    tokens= text.split()
    tokens = [token.strip() for token in tokens]
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text 

#### Method cleanSingleNews Takes a single news text and cleans it

In [6]:
def cleanSingleNews(text):
    text = removeStopWords(text)
    text = removePunctuation(text)
    return text

In [7]:
def cleanData(data):
    data =  [ [row[0], cleanSingleNews(row[1])]  for row in data]
    return data

In [8]:
train_data_cleaned = cleanData(train_data)
test_data_cleaned = cleanData(test_data)

In [9]:
#this statement takes a while to load, just holup
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary=True)

### News headline as a Average Vector 
_getSingleSentenceFeatures()_
- Ignores the word(s) those are unknown to model
- If all words in sentence are unknown, then return vector with all zero values

In [10]:
def getSingleSentenceFeatures(sentence):
    words = sentence.split()
    
    #check and remove if any word is not in models vocab
    words_filtered = [word for word in words if word in model.vocab ]
    
    #words removed from sentence
    '''
    print('Following Words Ignored from the sentence:')
    for word in words:
        if word not in words_filtered:
            print(word , end = ' ')
    '''
    
    
    if len(words_filtered) == 0: #rare case: if all words are unknown to model
        return np.zeros(300)

    features = model[words_filtered]
    #print((features.shape))
    avg_features = np.mean(features, axis=0)
    return avg_features

In [11]:
def buildFeatures(data):
    features_M =  [ [data_row[0], getSingleSentenceFeatures(data_row[1])] for data_row in data]
    return features_M

In [12]:
train_features = buildFeatures(train_data_cleaned)
test_features = buildFeatures(test_data_cleaned)

In [13]:
def printStats(algoResults):
    
    print('\nAlgorithm\t|TPs\t|TNs\t|FPs\t|FNs\t\t|F1-Score\t|Accuracy\t|Precision\t|Recall','\n'+'================================================================================================================')
    for algoResults_i in algoResults:
        confusionMatrix = algoResults_i[1]
        algoName = algoResults_i[0]
        
        tp = int(confusionMatrix[0][0])
        fp = int(confusionMatrix[0][1])
        fn = int(confusionMatrix[1][0])
        tn = int(confusionMatrix[1][1])


        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        accuracy = (tp+tn)/(tp+tn+fp+fn)
        f1Score = 2*((precision * recall) / (precision + recall))
        
        dec_fig = 3
        precision= round(precision, dec_fig)
        recall= round(recall, dec_fig)
        accuracy= round(accuracy,dec_fig )
        f1Score= round(f1Score, dec_fig)
        
        print(algoName+'\t|'+str(tp)+'\t|'+str(tn)+'\t|'+str(fp)+'\t|'+str(fn)+'\t\t|'+str(f1Score)+'   \t|'+str(accuracy)+'   \t|'+str(precision)+'   \t|'+str(recall))
        #print('------------------------------------------------------------------------------------------------------------------')

### Generic function for three algorithms

In [14]:
def getConfusionMatix(actualLabels, predictedLabels):
    confusionMatrix = np.zeros([2,2])
    for index, label in enumerate(actualLabels):
        goldLabel  = int(label)
        predictedLabel = int(predictedLabels[index])
        #if not goldLabel == predictedLabel:
        #print('goldLabel', goldLabel, "predictedLabel",predictedLabel,'@',index)
        if goldLabel == predictedLabel:
            #print("Correct Prediction")
            if(goldLabel):
                confusionMatrix[0][0] = confusionMatrix[0][0] +1
                #print("TP for")
            else:
                confusionMatrix[1][1] = confusionMatrix[1][1] +1
                #print("TN for")
        else:
            if(goldLabel):
                confusionMatrix[1][0] = confusionMatrix[1][0] +1
            else:
                confusionMatrix[0][1] = confusionMatrix[0][1] +1

    return confusionMatrix

In [15]:
def plotCosts(data, y_label, title):
    pyplot.plot(data, color='b')
    pyplot.xlabel('Iterations')
    pyplot.ylabel(y_label)
    pyplot.title(title)
    display.clear_output(wait=True)
    display.display(pyplot.gcf())
    time.sleep(0.2)

## Logistic Regression Implementation

In [16]:
def softmax(x, theta1, theta2):
    '''
    Function to calculate softmax of three hypotheses.
    Arguments
    ---------
    x : array
        A training example of shape n.
        
    theta1 : array
        The weights vector of shape (n+1,) for class 1.
        
    theta2 : array
        The weights vector of shape (n+1,) for class 2.
    
    Returns
    -------
    h_x : list
          A list containing the softmax of hypothesis of each class.

    '''
    h_x = list()
    x = np.append(1,x)
    z1 = sum(theta1 * x)
    z2 = sum(theta2 * x)
    

    Z = [z1, z2]
    Z_exp = np.exp(Z)
    h_x = Z_exp/np.sum(Z_exp)
    '''
    
    sum_exp = np.exp(z1) + np.exp(z2) 
    h1_x = np.exp(z1) / sum_exp
    h2_x = np.exp(z2) / sum_exp
    
    
   
    h_x = [h1_x, h2_x]
    
    #print(h_x, h_x_m)
    '''
    return h_x

In [17]:
def cross_entropy_loss(X, Y, theta1, theta2):
    '''
    This function calculates the cross-entropy-loss for multinomial logistic regression.
    
    Arguments
    ---------
    X : 2d-array
        The input dataset of shape (m, n), where m is the number of training examples and n is the number of features.
    
    Y : array
        The values of the function at each data point. This is a vector of
        shape (m, k), where m is the number of training examples and k is the number of categories.
    
    theta1 : array
        The weights vector of shape (n+1,) for class 1.
        
    theta2 : array
        The weights vector of shape (n+1,) for class 2.
    
    Returns
    -------
    J : array
        The value of multinomial the logistic regression cost function for each training example.

    '''
    # initialize some useful values
    m = X.shape[0] 
    n = X.shape[1] #number of features
    k = Y.shape[1]
    # You need to return the following variable(s) correctly
    J = 0
 
 
    
    hx = np.array([softmax(X[i,:], theta1, theta2) for i in range(0, m)])
    
    for i in range(0, k):
         J = J + ((-1/m)*sum(Y[:,i] * np.log(np.array(hx[:,i]))))
    return J

In [18]:
def batch_gradient_descent(X,X_with_X01, Y, alpha, n_epoch, batch_size):
    """
    Performs batch gradient descent to learn thetas. Updates thetas for each class simultaneously,  by taking `n_epoch`
    gradient steps with learning rate `alpha`.
    
    Arguments
    ---------
    X : 2d-array
        The input dataset of shape (m, n), where m is the number of training examples and n is the number of features.
    
    X_with_X01 : 2d-array
        The input dataset of shape (m+1, n), where m is the number of training examples and n is the number of features. Appending 1 to handle bias (X0 = 1)
    
    Y : array
        The values of the function at each data point. This is a vector of
        shape (m, k), where m is the number of training examples and k is the number of categories.
        
    alpha : float
        The learning rate.
    
    n_epoch : int
        The number of iterations for gradient descent. 
        
    batch_size :  int
        Size of the batch for Gradient Descent
    
    Returns
    -------
    theta1 : array
        The weights vector of shape (n+1,) for class 1.
        
    theta2 : array
        The weights vector of shape (n+1,) for class 2.
        
        
    J : list
        A python list for the values of the cost function after each iteration.
    
    """
    # initialize some useful values
    m = X.shape[0]  # number of training examples
    n = X.shape[1]
    J = list()  # list to store cost
    
    # You need to return the following variables correctly
    #, dtype=np.longdouble
    
    theta1 = np.zeros(n+1)
    theta2 = np.zeros(n+1)
    
    #X_with_X01 = np.concatenate((np.ones((m,1)),X), axis=1)
    for epoch in range(n_epoch):
        
        percent  = float("{0:.1f}".format((epoch/n_epoch)*100)) 
        ### START CODE HERE ### (≈ 5-10 lines of code)
        hx = np.array([softmax(X[i,:], theta1, theta2) for i in range(0, m)])
        for j in range(0, m, batch_size):
            sys.stdout.write("\rLogistic Regression: Epoch => "+str(percent)+"%"+"("+str(epoch+1)+"/"+str(n_epoch)+") \t Batches(size="+str(batch_size)+"): "+str(int(j/batch_size))+" / "+str(int(m/batch_size)))
            sys.stdout.flush()
            for k in range(0, n+1):
                temp1 = (alpha/batch_size) * sum((hx[j:j+batch_size,0]-Y[j:j+batch_size,0]) * X_with_X01[j:j+batch_size,k] )
                theta1[k] = theta1[k] - temp1
                
                temp2 = (alpha/batch_size) * sum((hx[j:j+batch_size,1]-Y[j:j+batch_size,1]) * X_with_X01[j:j+batch_size,k] )
                theta2[k] = theta2[k] - temp2
                
        ### END CODE HERE ###
        sys.stdout.write('\rCalculating Cost for current epoch........') #do not change this
        J.append(cross_entropy_loss(X, Y, theta1, theta2))
#         if(epoch>2):
#             plotCosts(J, 'Cost', 'Logistic Regression Cost')
    return theta1, theta2, J

In [19]:
def predict_logReg(X, theta1, theta2):
    '''
    Function which selects the most probable class by taking the max of probabilities.
    Arguments
    ---------
    X : 2d-array
        The test dataset of shape (m, n), where m is the number of instances and n is the number of features.
    
    theta1 : array
        The weights vector of shape (n+1,) for class 1.
        
    theta2 : array
        The weights vector of shape (n+1,) for class 2.
    
    Returns
    -------
    Y : array
        The predicted values of the function at each data point. This is a vector of
        shape (m, k), where m is the number of instances and k is the number of categories.
    

    '''
    m = X.shape[0]
    Y_predict = list()
    for i in range(0, m):
        h_x = softmax(X[i, :], theta1, theta2)
        max_arg = np.argmax(np.array(h_x))
        #print('max_arg',max_arg, h_x)
        if max_arg == 0:
            Y_predict.append([1, 0])
        else:
            Y_predict.append([0, 1])
    Y_predict = np.array(Y_predict)
    return Y_predict

In [20]:
def trainLogisticRegression(train_features, alpha,n_epoch, batch_size):

    Y_train = np.array(train_features)[:,0]
    X_train = np.array(train_features)[:,1]
    X_train = np.array([x for x in X_train])
    X_with_X01 = np.append(np.ones((len(X_train),1)), X_train , axis=1)
    Y = []
    for label_i in Y_train:
        if label_i == '0':
            Y.append([1,0])
        else: 
            Y.append([0,1])

    Y = np.array([y for y in Y])
    
    start = time.time()
    theta1, theta2, J1 = batch_gradient_descent(X_train, X_with_X01 ,Y, alpha,n_epoch, batch_size)
    end = time.time()
    print("\nLogistic Regression Training Completed in ", str(float("{0:.2f}".format((end-start)/60)))+" minute(s)")
    #print('Predicted theta = {}, cost = {}' .format (theta1, J1[-1]))
    return theta1, theta2, J1  

## KNN Implementation

In [21]:
def getCosineSimilarity(vector_train, vector_test):
    if(len(vector_train)!=len(vector_test)):
        print('Invalid Arguments')
        return 0
    denominator = (np.linalg.norm(vector_train)*np.linalg.norm(vector_test))
    if denominator == 0:
        #print("Returning 0 for cosine similarity")
        return 0
    cosine_similarity = np.dot(vector_train, vector_test)/denominator
    #print(cosine_similarity)
    return cosine_similarity

In [22]:
def getSimilarities_knn(train_data, test_data_i):
    similarities = []
    vector_test = test_data_i[1] 
    label_test = test_data_i[0]
   
    for train_data_i in train_data:
        vector_train = train_data_i[1]
        label_train = train_data_i[0]
        cosine_similarity = getCosineSimilarity(vector_train, vector_test)
        #print(distance)
        similarities.append([cosine_similarity, label_train])
    return similarities

In [23]:
def decideLabel_knn(knns , K):
    knns= np.array(knns)
    #print('knns',knns)
    all_labels = knns[:,1]
    #print("all_labels", all_labels, knns)
    mode_result = stats.mode(all_labels) 
    mode = mode_result[0] #mode at index zero 
    label = mode[0] #mode is returned as array
    return label
    

In [24]:
def getKNNs(train_data, test_data_i, K):
    #print(test_data_i) 
    similarities = getSimilarities_knn(train_data, test_data_i)
    
    similarities.sort(key=lambda x: x[0])
    
    knn = similarities[-K:]#distances[0:K]#
    return knn

In [25]:
def classify_knn(train_data, test_data_i, K):
    #print(test_data_i)
    k_neighbours = getKNNs(train_data, test_data_i, K)
    #print("KNNs",np.array(k_neighbours))
    label = decideLabel_knn(k_neighbours,K)
    #print(label)
    return label

## Perceptron Implementation

In [26]:
def change_labels(Y):
    '''
    Function changes labels from 0 and 1 to -1 and 1.
    Arguments
    ---------
    Y : array
        array with labels having values 0 and 1.
    
    Returns
    -------
    Y : array
        array with labels changed to 1 and -1.
    

    '''
    for i in range(0, len(Y)):
        Y[i] = int(Y[i])
        if Y[i] == 0:
            Y[i]= -1;
    return Y           

In [27]:
def perceptron_learning_algorithm(X, y, epoch):
    '''
    Function implements the perceptron learning algorithm. It updates weights when y(wx)<0.
    Arguments
    ---------
    X : 2d-array
        Input data of shape (m,n)
    y : array
        output labels of shape m.
    epoch : int
        number of times the perceptron learning algorithm runs.
    
    Returns
    -------
    w : array
        learned weights for perceptron learning algorithm.

    '''
    m = X.shape[0]
    n = X.shape[1]
    w = np.zeros(n) 
    errors = []
    for ep in range(0, epoch):
        percent  = float("{0:.1f}".format((ep/epoch)*100)) 
        error = 0
        for i in range(0, m):
            product = y[i]*sum(X[i, :]*w)
            if product <= 0:
                w = w + (y[i] * X[i, :])
                error = error + 1
        errors.append(error)
        sys.stdout.write("\rPerceptron Learning Epoch => "+str(percent)+"%"+"("+str(ep+1)+"/"+str(epoch)+")"+" \tErrors="+str(error))
        sys.stdout.flush()
#         if(ep>2 and ep%10==0):
#                plotCosts(errors,'Errors','Perceptron Weights Learning')
        if error == 0:
            break
    return w, errors

In [28]:
def predict_perceptron(X, w):
    '''
    predicts the output of perceptron learning algorithm.
    Arguments
    ---------
    X : 2d-array
        Input data of shape (m,n)
    w : array
        learned weights for perceptron learning algorithm.
    Returns
    -------
    y_predict : array
        predicted outputs for perceptron learning algorithm.

    '''
    #print(w)
    m = X.shape[0]
    n = X.shape[1]
    y_predict = []
    for i in range(0, m):
        product = sum(X[i, :]*w)
        if product < 0:
            y_predict.append(-1)
        else:
            y_predict.append(1)
    y_predict = np.array(y_predict)
    return y_predict

In [29]:
def decode_labels(y):
    '''
    changes the output labels back to 0 and 1 from -1 and 1.
    Arguments
    ---------
    y : array
        array with labels having values -1 and 1.
    Returns
    -------
    y : array
        array with labels having values 0 and 1.

    '''
    y_new = np.zeros(len(y))
    for i in range(0, len(y)):
        if y[i]==-1:
            y_new[i]=0
        else:
            y_new[i]=1
    return y_new

In [30]:
def executePerceptron(train_features, test_features, n_epoch):
    '''
    This function trains and tests the perceptron learning algorithm.
    Arguments
    ---------
    train_features : 2d-array
        array contains traing 
    train_features : 2d-array
        array with labels having values -1 and 1.
    train_features : 2d-array
        array with labels having values -1 and 1.
    Returns
    -------
    confusionMatrix : array
        array with labels having values 0 and 1.
    '''
    X_train = np.array([list(train_features[i][1]) for i in range(0, len(train_features))])
    Y_train = np.array([int(train_features[i][0]) for i in range(0, len(train_features))])
    Y_train = change_labels(Y_train)
    X_test = np.array([list(test_features[i][1]) for i in range(0, len(test_features))])
    Y_test = np.array([int(test_features[i][0]) for i in range(0, len(test_features))])
    Y_test = change_labels(Y_test)
    weights, errors = perceptron_learning_algorithm(X_train, Y_train, n_epoch)
    Y_hat = predict_perceptron(X_test, weights)
    
    actualLabels = decode_labels(Y_test)
    predictedLabels = decode_labels(Y_hat)
    confusionMatrix = getConfusionMatix(actualLabels, predictedLabels)
    
    return confusionMatrix

In [31]:
def executeKNN(train_features, test_features, K):
    #logic goes here
    
    
    t_featuers = test_features[0:300]
    
    actualLabels = np.array(t_featuers)[:,0]
    #print(actualLabels)
    predictedLabels = []
    testDataSize= len(t_featuers)
   
    for index, test_features_i in enumerate(t_featuers):
        percent  = float("{0:.1f}".format((index/testDataSize)*100)) 
        sys.stdout.write("\r"+str(K)+"-NN Tests =>\t "+str(percent)+"%"+"("+str(index+1)+"/"+str(testDataSize)+"Samples)")
        sys.stdout.flush()
        label = classify_knn(train_features, test_features_i, K)
        predictedLabels.append(label)
    #print(actualLabels, predictedLabels)
    confusionMatrix = getConfusionMatix(actualLabels, predictedLabels)
    return confusionMatrix  

In [32]:
def executeLogisticRegression(train_features, test_features, alpha,n_epoch, batch_size):
    theta1, theta2, J1 = trainLogisticRegression(train_features, alpha,n_epoch, batch_size)
    
    X_test = np.array(test_features)[:,1]
    X_test = np.array([x for x in X_test])
    X_test = np.array([x for x in X_test])
    actualLabels = np.array(test_features)[:,0]
    predictedLabels  = predict_logReg(X_test, theta1, theta2)
    predictedLabels = [0 if pl[0]==1 else 1 for pl in predictedLabels]
    confusionMatrix = getConfusionMatix(actualLabels, predictedLabels)
    return confusionMatrix

### This function executes 3 algorithm 

- Logistic Regression with Settings:
 -- Alpha = 0.001
 -- Iterations = 200
 -- Batch Size = 32
 
 
 - Kth Nearest Neighbour with: 
 -- K={1,3, 5, 7, 10}
 
 
 - Perceptron with:
 -- Iterations=500
 


In [33]:
def compareResults(train_features, test_features):
    
    algoResults = []
    
    alpha = 0.001
    n_epoch= 50
    batch_size =32
    confusionMatrix_lr = executeLogisticRegression(train_features, test_features,alpha,n_epoch, batch_size )
    algoResults.append(['LogisticReg', confusionMatrix_lr])
    
    
    
    Ks=[1, 3, 5, 7, 10]
    for K in Ks:
        confusionMatrix_knn = executeKNN(train_features, test_features, K)
        print('\n')#printing aline intentionally
        algoResults.append([str(K)+'-NN\t', confusionMatrix_knn])
    
    
    n_epoch_perceptron = 500
    confusionMatrix_perceptron = executePerceptron(train_features, test_features, n_epoch_perceptron)
    algoResults.append(['Perceptron', confusionMatrix_perceptron])
    
    return algoResults

In [34]:
algoResults = compareResults(train_features, test_features)

Calculating Cost for current epoch........ 	 Batches(size=32): 688 / 688
Logistic Regression Training Completed in  18.44 minute(s)
1-NN Tests =>	 99.7%(300/300Samples)

3-NN Tests =>	 99.7%(300/300Samples)

5-NN Tests =>	 99.7%(300/300Samples)

7-NN Tests =>	 99.7%(300/300Samples)

10-NN Tests =>	 99.7%(300/300Samples)

Perceptron Learning Epoch => 99.8%(500/500) 	Errors=7837

In [35]:
printStats(algoResults)


Algorithm	|TPs	|TNs	|FPs	|FNs		|F1-Score	|Accuracy	|Precision	|Recall 
LogisticReg	|1606	|2264	|733	|1121		|0.634   	|0.676   	|0.687   	|0.589
1-NN		|91	|122	|42	|45		|0.677   	|0.71   	|0.684   	|0.669
3-NN		|102	|121	|43	|34		|0.726   	|0.743   	|0.703   	|0.75
5-NN		|104	|121	|43	|32		|0.735   	|0.75   	|0.707   	|0.765
7-NN		|107	|125	|39	|29		|0.759   	|0.773   	|0.733   	|0.787
10-NN		|93	|138	|26	|43		|0.729   	|0.77   	|0.782   	|0.684
Perceptron	|2351	|1211	|1786	|376		|0.685   	|0.622   	|0.568   	|0.862
